In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA


In [26]:
loader = PyPDFDirectoryLoader("./us_census")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
final_documents = text_splitter.split_documents(documents)
len(final_documents)

293

In [27]:
from langchain_community.embeddings import HuggingFaceEmbeddings
# Embeddings using HuggingFace
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)


/var/folders/wx/rkj0bkkn1y55sq580xpv743h0000gn/T/ipykernel_2967/2104659727.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings = HuggingFaceEmbeddings(


In [29]:
import numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print( np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape )

[-5.03918827e-02 -1.84885710e-02 -1.61712859e-02  1.74641199e-02
  4.19471413e-02  3.85734327e-02  6.24263519e-03  4.36595716e-02
 -4.58340608e-02 -1.31456060e-02  4.47342545e-02  3.14254276e-02
 -1.56249683e-02 -8.44677910e-03 -4.92049800e-03  3.31841186e-02
 -5.69339748e-03  5.73156076e-03 -5.39106019e-02  3.83875296e-02
  7.59897335e-03  3.13764159e-03 -2.52936650e-02 -3.84591483e-02
  2.07458287e-02  1.94886532e-02 -1.26134707e-02 -1.31275430e-02
 -3.23045664e-02 -2.24549845e-01  1.08720856e-02  4.29711258e-03
 -1.79056320e-02 -1.29589383e-02 -6.92889048e-03 -5.24042780e-03
 -1.25167333e-02  7.02707395e-02  1.90644264e-02  3.14471573e-02
 -2.50507127e-02  2.98206843e-02 -3.96392448e-03 -1.71629675e-02
 -3.16858441e-02 -7.85869919e-03 -3.37052792e-02  1.69618230e-03
  1.10046472e-02 -2.17999853e-02  5.46133220e-02  1.15914874e-04
  3.66333015e-02  4.41419370e-02  3.08403559e-02 -3.03860847e-02
  2.04885285e-02 -2.97111878e-03 -6.22033933e-03  2.12305449e-02
  5.60942478e-02  1.09983

In [30]:
## Vectorstore creation
vectorstore = FAISS.from_documents(final_documents[:120], huggingface_embeddings)

In [31]:
query = "What is the health insurance coverage?"
relevant_documents = vectorstore.similarity_search(query)
print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [42]:
retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fa341607580> search_kwargs={'k': 3}


In [33]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [36]:
from langchain_community.llms import HuggingFaceHub
hugging_face = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hugging_face.invoke(query)

AttributeError: 'InferenceClient' object has no attribute 'post'

In [37]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0.1, "max_new_tokens": 300}
)
hf.invoke(query)

/var/folders/wx/rkj0bkkn1y55sq580xpv743h0000gn/T/ipykernel_2967/1388791672.py:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf = HuggingFacePipeline(


ValidationError: 1 validation error for HuggingFacePipeline
task
  Extra inputs are not permitted [type=extra_forbidden, input_value='text-generation', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [40]:
prompt_template = """
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
"""

In [41]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [46]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [48]:
query = "Differences in the uninsured rate by state in 2022"

In [50]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

AttributeError: 'InferenceClient' object has no attribute 'post'